In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SequentialFeatureSelector

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
text = pd.read_csv('/content/drive/MyDrive/학교/Dissertation/Dissertation_final/Data Preprocessing/all_features_scaled.csv')
text

,title,page_id,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,num_edits,num_editors,num_registered_editors,num_anonymous_editors,num_occasional_editors,edit_per_day,edit_per_editor,revert_count,discussion_count,quality
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,0.202138,0.020507,0.065268,-0.021923,0.032017,0.236123,1.059491,0.210672,0.171081,1
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,0.706190,0.543133,0.438547,0.622440,0.507665,0.775288,0.832776,0.779877,0.635288,1
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,0.125496,0.281341,0.104355,0.436776,0.319459,0.155290,-0.143283,-0.137176,-0.065684,1
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-0.428819,-0.416772,-0.294330,-0.517026,-0.407130,-0.422839,-0.232880,-0.401317,-0.218555,1
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,1.586678,1.760994,1.658053,1.798311,1.830809,1.527498,0.269954,1.507195,0.520634,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.669082,-0.726511,-0.765325,-0.666285,-0.735635,-0.677037,-0.678444,-0.555709,-0.257705,0
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.688688,-0.749526,-0.792686,-0.684488,-0.755026,-0.698471,-1.323641,-0.559429,-0.266094,0
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.684766,-0.744731,-0.786822,-0.680847,-0.751604,-0.693822,-1.131271,-0.557569,-0.266094,0
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.675142,-0.732265,-0.759462,-0.682668,-0.740197,-0.684439,-0.865996,-0.561290,-0.263298,0


In [12]:
doc2vec = pd.read_csv('/content/drive/MyDrive/학교/Dissertation/Dissertation_final/Feature Extraction/doc2vec_features.csv')
doc2vec

,title,page_id,doc2vec_20,doc2vec_50,doc2vec_100,doc2vec_200,doc2vec_300,doc2vec_500,quality
0,Mayan languages,182013,[-0.11873353 -2.4407356 -1.3702062 3.105298...,[ 0.14162295 -0.73532563 -1.8102438 -0.719203...,[ 0.21416852 -1.6696423 1.1597384 1.677749...,[-0.92931765 0.35427883 -0.985394 0.967223...,[-1.2080181 -0.70352554 -1.3096839 0.218799...,[-1.08123168e-01 -5.05179107e-01 -4.73065108e-...,1
1,Mu'awiya I,207068,[-2.07483 -1.4922532 0.36730346 3.651323...,[-0.94526184 -0.3110869 -1.9218819 -0.398054...,[-1.74806 -1.1632584 0.10795426 0.813521...,[-9.32636440e-01 -1.48135829e+00 1.86571255e-...,[ 0.1324497 -0.15933883 -0.56959057 0.640978...,[-1.1343669 -0.30720899 0.24345188 0.244366...,1
2,The Fountainhead,180464,[-0.939232 -2.142721 -1.883358 -0.108726...,[-0.89889634 -1.2338424 -0.1994613 1.074865...,[-1.2348349 -0.80121356 -0.1178944 -0.153580...,[-0.18963027 -0.5407947 0.02293755 0.160935...,[-3.95752490e-01 -2.72289246e-01 -4.50629920e-...,[ 4.51237231e-01 -3.93101126e-01 1.52836084e-...,1
3,Northern pintail,218361,[ 0.22895382 -0.07847051 -2.0693347 1.151979...,[-1.9152571 -0.9863042 -0.31720063 0.676733...,[-1.052102 -2.012778 0.20379645 -0.846639...,[-1.0868081 -1.1545188 -0.75203115 -0.079450...,[-0.15325828 -1.0265552 -0.950474 -0.276538...,[-2.55974084e-01 -1.00488138e+00 -7.53050864e-...,1
4,Manhattan Project,19603,[-0.41071716 -0.91254836 -3.3086228 0.386623...,[-0.08591695 -1.153746 -1.1509885 0.584178...,[-1.3185098 -1.2019124 0.5035422 0.113358...,[-1.0916361 -1.0232843 0.73881316 0.130395...,[ 1.85518786e-01 8.67399991e-01 -3.50051463e-...,[-2.31942884e-03 -4.61694479e-01 3.10965121e-...,1
...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,[-0.49925566 -0.5244077 -0.9469565 1.450050...,[-0.78056765 0.6843561 -1.1771741 0.382591...,[ 0.488909 -0.54169947 -0.3533258 0.162753...,[-2.64059007e-01 -4.53496762e-02 -6.67918384e-...,[-5.84085286e-01 6.87814727e-02 -3.03456843e-...,[-1.30619839e-01 -1.23517677e-01 2.14104261e-...,0
5196,Minawara and Multultu,95240,[-0.69623053 -0.28086147 -0.49423888 0.373790...,[-0.5233514 -0.5955352 -0.45576015 -0.262624...,[-3.48152429e-01 -5.32957375e-01 -1.07259654e-...,[-0.15931535 -0.45631117 -0.14412077 0.047031...,[-0.1366955 -0.18754531 -0.12493268 -0.018171...,[-1.55988351e-01 -9.39027667e-02 -5.48811592e-...,0
5197,Theophylline/ephedra/hydroxyzine,262652,[-0.55075014 -0.13917734 -0.75503784 0.880972...,[-0.5035148 -0.2839609 -0.3405831 0.169453...,[-2.67971367e-01 -4.94407803e-01 -1.63930863e-...,[-1.67261347e-01 -8.01966190e-02 -2.56586194e-...,[-1.96540684e-01 -6.82713836e-02 -1.28727436e-...,[-0.05836337 0.03332838 -0.02910833 0.038263...,0
5198,"Channel Lake, Illinois",111450,[-2.5712607 -0.99652314 -2.6514347 0.483577...,[-1.2923265 -0.7806829 -1.1711309 0.212995...,[-0.6354738 -0.79841405 0.05622012 0.378250...,[-9.76243198e-01 -6.22522295e-01 -4.92107987e-...,[-0.37194127 -0.10045881 -0.15332541 -0.275343...,[-1.61765054e-01 3.33138444e-02 -8.17700550e-...,0


In [13]:
# Function to process doc2vec columns
def clean_and_convert_vector(vector_str):
    # Remove newline characters and extra spaces
    cleaned_str = vector_str.replace('\n', ' ').replace('  ', ' ').strip()
    # Convert the cleaned string to a numpy array
    return np.fromstring(cleaned_str.strip('[]'), sep=' ')

In [7]:
def train_and_evaluate_models(X, y, cv=5, test_size=0.2):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Initialize the models
    models = {
        'Logistic Regression': LogisticRegression(),
        'SVM': SVC(),
        'Random Forest': RandomForestClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'XGBoost': XGBClassifier(),
        'MLP': MLPClassifier(),
        'KNN': KNeighborsClassifier()
    }

    # Initialize a list to store the results
    results = []

    # Define custom scoring
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score, average='weighted'),
        'recall': make_scorer(recall_score, average='weighted'),
        'f1': make_scorer(f1_score, average='weighted')
    }

    # StratifiedKFold for cross-validation
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    # Loop through the models
    for model_name, model in models.items():
        # Perform cross-validation on the training set
        cv_results = {metric: cross_val_score(model, X_train, y_train, cv=skf, scoring=scorer).mean() for metric, scorer in scoring.items()}

        # Train the model on the full training set
        model.fit(X_train, y_train)

        # Predict the test set
        y_pred = model.predict(X_test)

        # Calculate the metrics on the test set
        test_accuracy = accuracy_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred, average='weighted')
        test_recall = recall_score(y_test, y_pred, average='weighted')
        test_f1 = f1_score(y_test, y_pred, average='weighted')

        # Append the results to the list
        results.append({
            'Model': model_name,
            'CV Accuracy': cv_results['accuracy'],
            'CV Precision': cv_results['precision'],
            'CV Recall': cv_results['recall'],
            'CV F1 Score': cv_results['f1'],
            'Test Accuracy': test_accuracy,
            'Test Precision': test_precision,
            'Test Recall': test_recall,
            'Test F1 Score': test_f1
        })

    # Create a dataframe from the results
    results_df = pd.DataFrame(results)

    return results_df

In [14]:
doc2vec['doc2vec_20'] = doc2vec['doc2vec_20'].apply(clean_and_convert_vector)
doc2vec['doc2vec_50'] = doc2vec['doc2vec_50'].apply(clean_and_convert_vector)
doc2vec['doc2vec_100'] = doc2vec['doc2vec_100'].apply(clean_and_convert_vector)
doc2vec['doc2vec_200'] = doc2vec['doc2vec_200'].apply(clean_and_convert_vector)
doc2vec['doc2vec_300'] = doc2vec['doc2vec_300'].apply(clean_and_convert_vector)
doc2vec['doc2vec_500'] = doc2vec['doc2vec_500'].apply(clean_and_convert_vector)

In [15]:
doc2vec

,title,page_id,doc2vec_20,doc2vec_50,doc2vec_100,doc2vec_200,doc2vec_300,doc2vec_500,quality
0,Mayan languages,182013,"[-0.11873353, -2.4407356, -1.3702062, 3.105298...","[0.14162295, -0.73532563, -1.8102438, -0.71920...","[0.21416852, -1.6696423, 1.1597384, 1.6777499,...","[-0.92931765, 0.35427883, -0.985394, 0.9672233...","[-1.2080181, -0.70352554, -1.3096839, 0.218799...","[-0.108123168, -0.505179107, -0.473065108, 0.9...",1
1,Mu'awiya I,207068,"[-2.07483, -1.4922532, 0.36730346, 3.6513238, ...","[-0.94526184, -0.3110869, -1.9218819, -0.39805...","[-1.74806, -1.1632584, 0.10795426, 0.8135212, ...","[-0.93263644, -1.48135829, 0.186571255, 0.6616...","[0.1324497, -0.15933883, -0.56959057, 0.640978...","[-1.1343669, -0.30720899, 0.24345188, 0.244366...",1
2,The Fountainhead,180464,"[-0.939232, -2.142721, -1.883358, -0.10872623,...","[-0.89889634, -1.2338424, -0.1994613, 1.074865...","[-1.2348349, -0.80121356, -0.1178944, -0.15358...","[-0.18963027, -0.5407947, 0.02293755, 0.160935...","[-0.39575249, -0.272289246, -0.45062992, -1.13...","[0.451237231, -0.393101126, 0.152836084, -0.30...",1
3,Northern pintail,218361,"[0.22895382, -0.07847051, -2.0693347, 1.151979...","[-1.9152571, -0.9863042, -0.31720063, 0.676733...","[-1.052102, -2.012778, 0.20379645, -0.8466393,...","[-1.0868081, -1.1545188, -0.75203115, -0.07945...","[-0.15325828, -1.0265552, -0.950474, -0.276538...","[-0.255974084, -1.00488138, -0.753050864, 0.16...",1
4,Manhattan Project,19603,"[-0.41071716, -0.91254836, -3.3086228, 0.38662...","[-0.08591695, -1.153746, -1.1509885, 0.5841780...","[-1.3185098, -1.2019124, 0.5035422, 0.1133585,...","[-1.0916361, -1.0232843, 0.73881316, 0.1303952...","[0.185518786, 0.867399991, -0.350051463, 0.414...","[-0.00231942884, -0.461694479, 0.310965121, 0....",1
...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,"[-0.49925566, -0.5244077, -0.9469565, 1.450050...","[-0.78056765, 0.6843561, -1.1771741, 0.3825917...","[0.488909, -0.54169947, -0.3533258, 0.16275336...","[-0.264059007, -0.0453496762, -0.667918384, 0....","[-0.584085286, 0.0687814727, -0.303456843, -0....","[-0.130619839, -0.123517677, 0.00214104261, 0....",0
5196,Minawara and Multultu,95240,"[-0.69623053, -0.28086147, -0.49423888, 0.3737...","[-0.5233514, -0.5955352, -0.45576015, -0.26262...","[-0.348152429, -0.532957375, -0.107259654, 0.1...","[-0.15931535, -0.45631117, -0.14412077, 0.0470...","[-0.1366955, -0.18754531, -0.12493268, -0.0181...","[-0.155988351, -0.0939027667, -0.0548811592, 0...",0
5197,Theophylline/ephedra/hydroxyzine,262652,"[-0.55075014, -0.13917734, -0.75503784, 0.8809...","[-0.5035148, -0.2839609, -0.3405831, 0.1694538...","[-0.267971367, -0.494407803, -0.163930863, 0.0...","[-0.167261347, -0.080196619, -0.256586194, 0.2...","[-0.196540684, -0.0682713836, -0.128727436, 0....","[-0.05836337, 0.03332838, -0.02910833, 0.03826...",0
5198,"Channel Lake, Illinois",111450,"[-2.5712607, -0.99652314, -2.6514347, 0.483577...","[-1.2923265, -0.7806829, -1.1711309, 0.2129957...","[-0.6354738, -0.79841405, 0.05622012, 0.378250...","[-0.976243198, -0.622522295, -0.492107987, 0.1...","[-0.37194127, -0.10045881, -0.15332541, -0.275...","[-0.161765054, 0.0333138444, -0.081770055, 0.1...",0


### Combining all text features and doc2vec

__Combining all text features and doc2vec_20__

In [17]:
# Create features using doc2vec_20
df_20 = doc2vec.copy()
df_20 = df_20[['title', 'page_id', 'doc2vec_20', 'quality']]

vector_columns = pd.DataFrame(df_20['doc2vec_20'].tolist(), columns=[f'var_{i+1}' for i in range(20)])
df_20 = df_20.join(vector_columns)
df_20

,title,page_id,doc2vec_20,quality,var_1,var_2,var_3,var_4,var_5,var_6,...,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20
0,Mayan languages,182013,"[-0.11873353, -2.4407356, -1.3702062, 3.105298...",1,-0.118734,-2.440736,-1.370206,3.105298,0.444223,1.270455,...,2.589589,-3.304577,-0.880189,0.132373,-1.697009,-1.738696,0.542941,0.903049,-1.734197,0.417213
1,Mu'awiya I,207068,"[-2.07483, -1.4922532, 0.36730346, 3.6513238, ...",1,-2.074830,-1.492253,0.367303,3.651324,1.235692,-0.449774,...,1.648793,0.241381,-0.366885,-0.707308,-2.055417,-1.595411,1.199849,-2.256915,-1.318837,-0.628533
2,The Fountainhead,180464,"[-0.939232, -2.142721, -1.883358, -0.10872623,...",1,-0.939232,-2.142721,-1.883358,-0.108726,1.719563,-1.276436,...,-0.990515,0.894514,-0.850906,-1.573790,-0.939912,0.271126,0.087762,0.493070,-0.744864,0.943934
3,Northern pintail,218361,"[0.22895382, -0.07847051, -2.0693347, 1.151979...",1,0.228954,-0.078471,-2.069335,1.151980,-0.904061,-0.028492,...,-0.217106,-0.038556,-2.535328,-1.079626,0.984010,-3.052293,-1.283746,0.150716,-2.270509,0.511353
4,Manhattan Project,19603,"[-0.41071716, -0.91254836, -3.3086228, 0.38662...",1,-0.410717,-0.912548,-3.308623,0.386624,2.255983,0.510913,...,-0.649519,1.573444,0.557071,-1.295349,1.095210,-0.513653,-0.424813,0.191953,-2.206298,-0.896591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,"[-0.49925566, -0.5244077, -0.9469565, 1.450050...",0,-0.499256,-0.524408,-0.946956,1.450051,0.232176,0.072388,...,1.364483,1.565599,-0.117768,-0.553074,-1.408595,-1.481983,0.412338,-0.264874,-1.365068,-1.363219
5196,Minawara and Multultu,95240,"[-0.69623053, -0.28086147, -0.49423888, 0.3737...",0,-0.696231,-0.280861,-0.494239,0.373790,0.157850,0.087360,...,-0.029130,0.056235,-1.061213,0.044772,-0.044322,-0.851839,-0.056814,0.286022,-0.846064,-0.032695
5197,Theophylline/ephedra/hydroxyzine,262652,"[-0.55075014, -0.13917734, -0.75503784, 0.8809...",0,-0.550750,-0.139177,-0.755038,0.880972,0.098353,0.048484,...,0.014874,0.138150,-0.292356,-0.286313,-0.223987,-0.202073,-0.371795,0.419180,-0.500988,-0.126529
5198,"Channel Lake, Illinois",111450,"[-2.5712607, -0.99652314, -2.6514347, 0.483577...",0,-2.571261,-0.996523,-2.651435,0.483577,0.259311,-0.176672,...,1.413022,1.097628,-1.258792,0.260787,-0.695690,-1.898501,-1.764783,0.649597,-1.145714,-0.401934


In [18]:
# Combine all text features with doc2vec_20
combined_20 = text.merge(df_20, on='title')
combined_20

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,2.589589,-3.304577,-0.880189,0.132373,-1.697009,-1.738696,0.542941,0.903049,-1.734197,0.417213
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,1.648793,0.241381,-0.366885,-0.707308,-2.055417,-1.595411,1.199849,-2.256915,-1.318837,-0.628533
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.990515,0.894514,-0.850906,-1.573790,-0.939912,0.271126,0.087762,0.493070,-0.744864,0.943934
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-0.217106,-0.038556,-2.535328,-1.079626,0.984010,-3.052293,-1.283746,0.150716,-2.270509,0.511353
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.649519,1.573444,0.557071,-1.295349,1.095210,-0.513653,-0.424813,0.191953,-2.206298,-0.896591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,1.364483,1.565599,-0.117768,-0.553074,-1.408595,-1.481983,0.412338,-0.264874,-1.365068,-1.363219
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.029130,0.056235,-1.061213,0.044772,-0.044322,-0.851839,-0.056814,0.286022,-0.846064,-0.032695
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,0.014874,0.138150,-0.292356,-0.286313,-0.223987,-0.202073,-0.371795,0.419180,-0.500988,-0.126529
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,1.413022,1.097628,-1.258792,0.260787,-0.695690,-1.898501,-1.764783,0.649597,-1.145714,-0.401934


In [19]:
combined_20[combined_20['page_id_x']!=combined_20['page_id_y']]

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20


In [20]:
combined_20.drop(['page_id_y', 'quality_y', 'doc2vec_20'], axis=1, inplace=True)
combined_20.rename(columns={'page_id_x': 'page_id', 'quality_x': 'quality'}, inplace=True)
combined_20

,title,page_id,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,2.589589,-3.304577,-0.880189,0.132373,-1.697009,-1.738696,0.542941,0.903049,-1.734197,0.417213
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,1.648793,0.241381,-0.366885,-0.707308,-2.055417,-1.595411,1.199849,-2.256915,-1.318837,-0.628533
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.990515,0.894514,-0.850906,-1.573790,-0.939912,0.271126,0.087762,0.493070,-0.744864,0.943934
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-0.217106,-0.038556,-2.535328,-1.079626,0.984010,-3.052293,-1.283746,0.150716,-2.270509,0.511353
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.649519,1.573444,0.557071,-1.295349,1.095210,-0.513653,-0.424813,0.191953,-2.206298,-0.896591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,1.364483,1.565599,-0.117768,-0.553074,-1.408595,-1.481983,0.412338,-0.264874,-1.365068,-1.363219
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.029130,0.056235,-1.061213,0.044772,-0.044322,-0.851839,-0.056814,0.286022,-0.846064,-0.032695
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,0.014874,0.138150,-0.292356,-0.286313,-0.223987,-0.202073,-0.371795,0.419180,-0.500988,-0.126529
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,1.413022,1.097628,-1.258792,0.260787,-0.695690,-1.898501,-1.764783,0.649597,-1.145714,-0.401934


In [22]:
X = combined_20.drop(['title', 'page_id', 'quality'], axis=1)
y = combined_20['quality']

result_20 = train_and_evaluate_models(X, y)
result_20

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.890625,0.891054,0.890625,0.890595,0.905769,0.906370,0.905769,0.905734
1,SVM,0.900240,0.901265,0.900240,0.900176,0.913462,0.914344,0.913462,0.913415
2,Random Forest,0.892548,0.895521,0.890865,0.893952,0.898077,0.898554,0.898077,0.898046
3,AdaBoost,0.890625,0.891243,0.890625,0.890579,0.888462,0.888743,0.888462,0.888441
4,XGBoost,0.908413,0.909239,0.908413,0.908370,0.913462,0.913615,0.913462,0.913454
5,MLP,0.893990,0.898816,0.891106,0.892028,0.888462,0.888513,0.888462,0.888458
6,KNN,0.861298,0.864343,0.861298,0.861007,0.876923,0.878738,0.876923,0.876775


__Combining all text features and doc2vec_50__

In [23]:
# Create features using doc2vec_50
df_50 = doc2vec.copy()
df_50 = df_50[['title', 'page_id', 'doc2vec_50', 'quality']]

vector_columns = pd.DataFrame(df_50['doc2vec_50'].tolist(), columns=[f'var_{i+1}' for i in range(50)])
df_50 = df_50.join(vector_columns)
df_50

,title,page_id,doc2vec_50,quality,var_1,var_2,var_3,var_4,var_5,var_6,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,Mayan languages,182013,"[0.14162295, -0.73532563, -1.8102438, -0.71920...",1,0.141623,-0.735326,-1.810244,-0.719204,1.129141,-0.576397,...,-0.609255,1.594196,1.374301,0.580563,0.753562,-0.446759,1.098445,2.162488,0.269427,-2.083542
1,Mu'awiya I,207068,"[-0.94526184, -0.3110869, -1.9218819, -0.39805...",1,-0.945262,-0.311087,-1.921882,-0.398054,-0.152138,2.213863,...,0.664694,-0.060651,0.799270,-0.958273,0.337148,1.416505,2.117402,-0.944979,2.191666,-0.045392
2,The Fountainhead,180464,"[-0.89889634, -1.2338424, -0.1994613, 1.074865...",1,-0.898896,-1.233842,-0.199461,1.074866,-0.870027,0.429251,...,0.899741,-0.025535,0.509132,-1.556558,-1.069190,0.774159,0.017817,1.039071,0.768530,-1.704601
3,Northern pintail,218361,"[-1.9152571, -0.9863042, -0.31720063, 0.676733...",1,-1.915257,-0.986304,-0.317201,0.676734,0.730094,1.212762,...,0.042587,-0.301828,-0.268440,-1.096084,0.953988,1.231612,0.365509,0.223079,0.547631,-0.617974
4,Manhattan Project,19603,"[-0.08591695, -1.153746, -1.1509885, 0.5841780...",1,-0.085917,-1.153746,-1.150988,0.584178,1.211792,-1.038027,...,0.606108,1.389589,1.016950,-0.579469,0.645308,1.044399,-0.144792,1.077177,-1.111737,-1.869863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,"[-0.78056765, 0.6843561, -1.1771741, 0.3825917...",0,-0.780568,0.684356,-1.177174,0.382592,-0.185709,0.490036,...,-0.075037,-0.018248,0.332292,-0.523666,-0.004916,0.925149,-0.161927,0.669919,-0.779033,-0.517922
5196,Minawara and Multultu,95240,"[-0.5233514, -0.5955352, -0.45576015, -0.26262...",0,-0.523351,-0.595535,-0.455760,-0.262625,-0.085185,0.109685,...,0.147598,-0.126686,0.140621,-0.720846,0.455311,-0.059910,0.031989,0.164125,0.189279,-0.170787
5197,Theophylline/ephedra/hydroxyzine,262652,"[-0.5035148, -0.2839609, -0.3405831, 0.1694538...",0,-0.503515,-0.283961,-0.340583,0.169454,-0.049155,0.173916,...,0.037891,-0.399214,0.291244,-0.123542,0.090050,-0.166694,-0.044172,0.010083,0.038130,-0.616626
5198,"Channel Lake, Illinois",111450,"[-1.2923265, -0.7806829, -1.1711309, 0.2129957...",0,-1.292326,-0.780683,-1.171131,0.212996,0.607244,0.479706,...,-0.256851,-0.634215,-0.124935,-0.202607,0.482201,-0.721821,-0.682262,0.804415,0.242318,-1.592898


In [24]:
# Combine all text features with doc2vec_50
combined_50 = text.merge(df_50, on='title')
combined_50

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.609255,1.594196,1.374301,0.580563,0.753562,-0.446759,1.098445,2.162488,0.269427,-2.083542
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,0.664694,-0.060651,0.799270,-0.958273,0.337148,1.416505,2.117402,-0.944979,2.191666,-0.045392
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,0.899741,-0.025535,0.509132,-1.556558,-1.069190,0.774159,0.017817,1.039071,0.768530,-1.704601
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,0.042587,-0.301828,-0.268440,-1.096084,0.953988,1.231612,0.365509,0.223079,0.547631,-0.617974
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,0.606108,1.389589,1.016950,-0.579469,0.645308,1.044399,-0.144792,1.077177,-1.111737,-1.869863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.075037,-0.018248,0.332292,-0.523666,-0.004916,0.925149,-0.161927,0.669919,-0.779033,-0.517922
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,0.147598,-0.126686,0.140621,-0.720846,0.455311,-0.059910,0.031989,0.164125,0.189279,-0.170787
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,0.037891,-0.399214,0.291244,-0.123542,0.090050,-0.166694,-0.044172,0.010083,0.038130,-0.616626
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.256851,-0.634215,-0.124935,-0.202607,0.482201,-0.721821,-0.682262,0.804415,0.242318,-1.592898


In [25]:
combined_50[combined_50['page_id_x']!=combined_50['page_id_y']]

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50


In [26]:
combined_50.drop(['page_id_y', 'quality_y', 'doc2vec_50'], axis=1, inplace=True)
combined_50.rename(columns={'page_id_x': 'page_id', 'quality_x': 'quality'}, inplace=True)
combined_50

,title,page_id,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.609255,1.594196,1.374301,0.580563,0.753562,-0.446759,1.098445,2.162488,0.269427,-2.083542
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,0.664694,-0.060651,0.799270,-0.958273,0.337148,1.416505,2.117402,-0.944979,2.191666,-0.045392
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,0.899741,-0.025535,0.509132,-1.556558,-1.069190,0.774159,0.017817,1.039071,0.768530,-1.704601
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,0.042587,-0.301828,-0.268440,-1.096084,0.953988,1.231612,0.365509,0.223079,0.547631,-0.617974
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,0.606108,1.389589,1.016950,-0.579469,0.645308,1.044399,-0.144792,1.077177,-1.111737,-1.869863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.075037,-0.018248,0.332292,-0.523666,-0.004916,0.925149,-0.161927,0.669919,-0.779033,-0.517922
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,0.147598,-0.126686,0.140621,-0.720846,0.455311,-0.059910,0.031989,0.164125,0.189279,-0.170787
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,0.037891,-0.399214,0.291244,-0.123542,0.090050,-0.166694,-0.044172,0.010083,0.038130,-0.616626
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.256851,-0.634215,-0.124935,-0.202607,0.482201,-0.721821,-0.682262,0.804415,0.242318,-1.592898


In [27]:
X = combined_50.drop(['title', 'page_id', 'quality'], axis=1)
y = combined_50['quality']

result_50 = train_and_evaluate_models(X, y)
result_50

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.888462,0.888832,0.888462,0.888435,0.904808,0.905241,0.904808,0.904782
1,SVM,0.902163,0.903317,0.902163,0.902090,0.914423,0.915235,0.914423,0.914381
2,Random Forest,0.893029,0.896595,0.893990,0.891500,0.896154,0.896629,0.896154,0.896123
3,AdaBoost,0.889423,0.889898,0.889423,0.889389,0.872115,0.872514,0.872115,0.872081
4,XGBoost,0.903846,0.904779,0.903846,0.903788,0.906731,0.906985,0.906731,0.906716
5,MLP,0.902885,0.899415,0.904808,0.905726,0.908654,0.908728,0.908654,0.908650
6,KNN,0.867548,0.869645,0.867548,0.867361,0.875962,0.877482,0.875962,0.875837


__Combining all text features and doc2vec_100__

In [28]:
# Create features using doc2vec_100
df_100 = doc2vec.copy()
df_100 = df_100[['title', 'page_id', 'doc2vec_100', 'quality']]

vector_columns = pd.DataFrame(df_100['doc2vec_100'].tolist(), columns=[f'var_{i+1}' for i in range(100)])
df_100 = df_100.join(vector_columns)
df_100

,title,page_id,doc2vec_100,quality,var_1,var_2,var_3,var_4,var_5,var_6,...,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100
0,Mayan languages,182013,"[0.21416852, -1.6696423, 1.1597384, 1.6777499,...",1,0.214169,-1.669642,1.159738,1.677750,1.294358,-0.428283,...,-0.780045,0.126435,0.457684,-0.976941,-0.033570,0.909304,-0.341177,-0.686826,0.434214,-0.085368
1,Mu'awiya I,207068,"[-1.74806, -1.1632584, 0.10795426, 0.8135212, ...",1,-1.748060,-1.163258,0.107954,0.813521,1.220592,0.360374,...,-0.221565,1.052404,-0.155383,0.833835,-0.108549,2.475773,1.293293,-0.061450,0.716968,0.258674
2,The Fountainhead,180464,"[-1.2348349, -0.80121356, -0.1178944, -0.15358...",1,-1.234835,-0.801214,-0.117894,-0.153581,1.144619,-0.161707,...,-0.328424,0.488366,-1.207654,0.679866,0.896923,-0.140029,-0.371535,-0.175522,1.484022,-0.286713
3,Northern pintail,218361,"[-1.052102, -2.012778, 0.20379645, -0.8466393,...",1,-1.052102,-2.012778,0.203796,-0.846639,0.267861,0.168490,...,-0.514490,0.849899,-0.754519,-1.038317,0.780983,-1.046274,-1.555287,-0.416799,0.093230,-0.191590
4,Manhattan Project,19603,"[-1.3185098, -1.2019124, 0.5035422, 0.1133585,...",1,-1.318510,-1.201912,0.503542,0.113359,-0.168953,-0.212796,...,-0.186288,-0.263700,0.289110,0.183705,-0.029313,0.589104,-1.494260,-0.030581,0.671126,2.263582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,"[0.488909, -0.54169947, -0.3533258, 0.16275336...",0,0.488909,-0.541699,-0.353326,0.162753,1.301805,0.711333,...,-0.559690,0.546811,-0.635086,0.178669,0.045058,0.377926,0.065588,-0.109098,1.108363,0.593977
5196,Minawara and Multultu,95240,"[-0.348152429, -0.532957375, -0.107259654, 0.1...",0,-0.348152,-0.532957,-0.107260,0.169512,0.235168,0.055693,...,-0.222619,0.109795,-0.336194,-0.337256,-0.021674,0.307858,-0.396411,-0.137848,0.027160,-0.120415
5197,Theophylline/ephedra/hydroxyzine,262652,"[-0.267971367, -0.494407803, -0.163930863, 0.0...",0,-0.267971,-0.494408,-0.163931,0.081484,0.302020,-0.009572,...,-0.165442,-0.281676,-0.094921,-0.101845,-0.039904,0.236989,-0.154063,-0.092468,0.126725,0.089715
5198,"Channel Lake, Illinois",111450,"[-0.6354738, -0.79841405, 0.05622012, 0.378250...",0,-0.635474,-0.798414,0.056220,0.378250,0.377347,-0.031710,...,0.176425,0.115690,-0.924295,-0.120078,-0.240744,0.456143,-0.160728,-0.819485,0.037214,0.470418


In [29]:
# Combine all text features with doc2vec_100
combined_100 = text.merge(df_100, on='title')
combined_100

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.780045,0.126435,0.457684,-0.976941,-0.033570,0.909304,-0.341177,-0.686826,0.434214,-0.085368
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,-0.221565,1.052404,-0.155383,0.833835,-0.108549,2.475773,1.293293,-0.061450,0.716968,0.258674
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.328424,0.488366,-1.207654,0.679866,0.896923,-0.140029,-0.371535,-0.175522,1.484022,-0.286713
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-0.514490,0.849899,-0.754519,-1.038317,0.780983,-1.046274,-1.555287,-0.416799,0.093230,-0.191590
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.186288,-0.263700,0.289110,0.183705,-0.029313,0.589104,-1.494260,-0.030581,0.671126,2.263582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.559690,0.546811,-0.635086,0.178669,0.045058,0.377926,0.065588,-0.109098,1.108363,0.593977
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.222619,0.109795,-0.336194,-0.337256,-0.021674,0.307858,-0.396411,-0.137848,0.027160,-0.120415
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.165442,-0.281676,-0.094921,-0.101845,-0.039904,0.236989,-0.154063,-0.092468,0.126725,0.089715
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,0.176425,0.115690,-0.924295,-0.120078,-0.240744,0.456143,-0.160728,-0.819485,0.037214,0.470418


In [30]:
combined_100[combined_100['page_id_x']!=combined_100['page_id_y']]

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100


In [31]:
combined_100.drop(['page_id_y', 'quality_y', 'doc2vec_100'], axis=1, inplace=True)
combined_100.rename(columns={'page_id_x': 'page_id', 'quality_x': 'quality'}, inplace=True)
combined_100

,title,page_id,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.780045,0.126435,0.457684,-0.976941,-0.033570,0.909304,-0.341177,-0.686826,0.434214,-0.085368
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,-0.221565,1.052404,-0.155383,0.833835,-0.108549,2.475773,1.293293,-0.061450,0.716968,0.258674
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.328424,0.488366,-1.207654,0.679866,0.896923,-0.140029,-0.371535,-0.175522,1.484022,-0.286713
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-0.514490,0.849899,-0.754519,-1.038317,0.780983,-1.046274,-1.555287,-0.416799,0.093230,-0.191590
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.186288,-0.263700,0.289110,0.183705,-0.029313,0.589104,-1.494260,-0.030581,0.671126,2.263582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.559690,0.546811,-0.635086,0.178669,0.045058,0.377926,0.065588,-0.109098,1.108363,0.593977
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.222619,0.109795,-0.336194,-0.337256,-0.021674,0.307858,-0.396411,-0.137848,0.027160,-0.120415
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.165442,-0.281676,-0.094921,-0.101845,-0.039904,0.236989,-0.154063,-0.092468,0.126725,0.089715
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,0.176425,0.115690,-0.924295,-0.120078,-0.240744,0.456143,-0.160728,-0.819485,0.037214,0.470418


In [32]:
X = combined_100.drop(['title', 'page_id', 'quality'], axis=1)
y = combined_100['quality']

result_100 = train_and_evaluate_models(X, y)
result_100

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.887260,0.887610,0.887260,0.887232,0.901923,0.902405,0.901923,0.901894
1,SVM,0.900240,0.901266,0.900240,0.900175,0.912500,0.913971,0.912500,0.912422
2,Random Forest,0.889904,0.891827,0.890144,0.885011,0.900000,0.901163,0.900000,0.899927
3,AdaBoost,0.881010,0.882081,0.881010,0.880921,0.890385,0.890668,0.890385,0.890365
4,XGBoost,0.907452,0.908420,0.907452,0.907396,0.903846,0.904061,0.903846,0.903833
5,MLP,0.897596,0.895390,0.900962,0.898758,0.903846,0.903996,0.903846,0.903837
6,KNN,0.865144,0.865654,0.865144,0.865099,0.875000,0.875022,0.875000,0.874998


__Combining all text features and doc2vec_200__

In [33]:
# Create features using doc2vec_200
df_200 = doc2vec.copy()
df_200 = df_200[['title', 'page_id', 'doc2vec_200', 'quality']]

vector_columns = pd.DataFrame(df_200['doc2vec_200'].tolist(), columns=[f'var_{i+1}' for i in range(200)])
df_200 = df_200.join(vector_columns)
df_200

,title,page_id,doc2vec_200,quality,var_1,var_2,var_3,var_4,var_5,var_6,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
0,Mayan languages,182013,"[-0.92931765, 0.35427883, -0.985394, 0.9672233...",1,-0.929318,0.354279,-0.985394,0.967223,0.281792,-0.142656,...,-0.106629,1.090708,1.461626,0.955887,-0.202579,-0.573331,0.866143,0.616094,1.081148,0.430580
1,Mu'awiya I,207068,"[-0.93263644, -1.48135829, 0.186571255, 0.6616...",1,-0.932636,-1.481358,0.186571,0.661641,0.928167,-0.541309,...,0.162602,-0.216309,0.575019,-0.115410,-0.504757,0.009841,0.690092,-0.029063,-0.883779,0.688236
2,The Fountainhead,180464,"[-0.18963027, -0.5407947, 0.02293755, 0.160935...",1,-0.189630,-0.540795,0.022938,0.160936,-0.444382,-0.207634,...,-0.620485,0.021705,0.489078,-0.223216,-0.575344,-0.411644,0.330804,-0.666717,0.395511,0.491348
3,Northern pintail,218361,"[-1.0868081, -1.1545188, -0.75203115, -0.07945...",1,-1.086808,-1.154519,-0.752031,-0.079451,-0.022155,0.186989,...,-1.225472,0.418796,0.110502,0.687321,-0.788272,0.552987,0.750681,-1.376309,-0.622652,0.832225
4,Manhattan Project,19603,"[-1.0916361, -1.0232843, 0.73881316, 0.1303952...",1,-1.091636,-1.023284,0.738813,0.130395,-0.390660,0.356297,...,-0.478512,-0.607274,-0.141831,0.395690,0.225157,-0.350576,-0.720165,-0.330573,-0.203796,0.654384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,"[-0.264059007, -0.0453496762, -0.667918384, 0....",0,-0.264059,-0.045350,-0.667918,0.495617,0.074979,0.173139,...,0.173049,0.041855,0.120512,-0.134590,-0.632679,0.220900,-0.002116,-0.202400,0.060884,0.084081
5196,Minawara and Multultu,95240,"[-0.15931535, -0.45631117, -0.14412077, 0.0470...",0,-0.159315,-0.456311,-0.144121,0.047031,-0.137587,-0.037770,...,-0.340014,0.092110,0.237233,0.201323,0.036695,0.055461,0.126255,-0.208947,-0.118132,-0.020965
5197,Theophylline/ephedra/hydroxyzine,262652,"[-0.167261347, -0.080196619, -0.256586194, 0.2...",0,-0.167261,-0.080197,-0.256586,0.253466,-0.068370,-0.058681,...,-0.153126,-0.003978,0.150722,0.290779,-0.041975,0.130378,0.077555,-0.192577,-0.280983,0.013695
5198,"Channel Lake, Illinois",111450,"[-0.976243198, -0.622522295, -0.492107987, 0.1...",0,-0.976243,-0.622522,-0.492108,0.119893,-0.371019,0.443693,...,-0.437081,0.405612,0.316170,0.677957,-0.330192,0.161981,-0.006467,-0.085397,-0.287258,0.307054


In [34]:
# Combine all text features with doc2vec_200
combined_200 = text.merge(df_200, on='title')
combined_200

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.106629,1.090708,1.461626,0.955887,-0.202579,-0.573331,0.866143,0.616094,1.081148,0.430580
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,0.162602,-0.216309,0.575019,-0.115410,-0.504757,0.009841,0.690092,-0.029063,-0.883779,0.688236
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.620485,0.021705,0.489078,-0.223216,-0.575344,-0.411644,0.330804,-0.666717,0.395511,0.491348
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-1.225472,0.418796,0.110502,0.687321,-0.788272,0.552987,0.750681,-1.376309,-0.622652,0.832225
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.478512,-0.607274,-0.141831,0.395690,0.225157,-0.350576,-0.720165,-0.330573,-0.203796,0.654384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,0.173049,0.041855,0.120512,-0.134590,-0.632679,0.220900,-0.002116,-0.202400,0.060884,0.084081
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.340014,0.092110,0.237233,0.201323,0.036695,0.055461,0.126255,-0.208947,-0.118132,-0.020965
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.153126,-0.003978,0.150722,0.290779,-0.041975,0.130378,0.077555,-0.192577,-0.280983,0.013695
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.437081,0.405612,0.316170,0.677957,-0.330192,0.161981,-0.006467,-0.085397,-0.287258,0.307054


In [35]:
combined_200[combined_200['page_id_x']!=combined_200['page_id_y']]

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200


In [36]:
combined_200.drop(['page_id_y', 'quality_y', 'doc2vec_200'], axis=1, inplace=True)
combined_200.rename(columns={'page_id_x': 'page_id', 'quality_x': 'quality'}, inplace=True)
combined_200

,title,page_id,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.106629,1.090708,1.461626,0.955887,-0.202579,-0.573331,0.866143,0.616094,1.081148,0.430580
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,0.162602,-0.216309,0.575019,-0.115410,-0.504757,0.009841,0.690092,-0.029063,-0.883779,0.688236
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.620485,0.021705,0.489078,-0.223216,-0.575344,-0.411644,0.330804,-0.666717,0.395511,0.491348
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-1.225472,0.418796,0.110502,0.687321,-0.788272,0.552987,0.750681,-1.376309,-0.622652,0.832225
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.478512,-0.607274,-0.141831,0.395690,0.225157,-0.350576,-0.720165,-0.330573,-0.203796,0.654384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,0.173049,0.041855,0.120512,-0.134590,-0.632679,0.220900,-0.002116,-0.202400,0.060884,0.084081
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.340014,0.092110,0.237233,0.201323,0.036695,0.055461,0.126255,-0.208947,-0.118132,-0.020965
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.153126,-0.003978,0.150722,0.290779,-0.041975,0.130378,0.077555,-0.192577,-0.280983,0.013695
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.437081,0.405612,0.316170,0.677957,-0.330192,0.161981,-0.006467,-0.085397,-0.287258,0.307054


In [37]:
X = combined_200.drop(['title', 'page_id', 'quality'], axis=1)
y = combined_200['quality']

result_200 = train_and_evaluate_models(X, y)
result_200

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.876442,0.876791,0.876442,0.876413,0.890385,0.890963,0.890385,0.890344
1,SVM,0.896635,0.897916,0.896635,0.896549,0.909615,0.910806,0.909615,0.909550
2,Random Forest,0.887260,0.885284,0.887981,0.885425,0.888462,0.890129,0.888462,0.888342
3,AdaBoost,0.875962,0.876601,0.875962,0.875907,0.885577,0.885647,0.885577,0.885572
4,XGBoost,0.900721,0.901926,0.900721,0.900646,0.898077,0.898791,0.898077,0.898031
5,MLP,0.887740,0.892470,0.889904,0.888878,0.907692,0.908714,0.907692,0.907635
6,KNN,0.861058,0.862126,0.861058,0.860950,0.858654,0.859773,0.858654,0.858544


__Combining all text features and doc2vec_300__

In [38]:
# Create features using doc2vec_300
df_300 = doc2vec.copy()
df_300 = df_300[['title', 'page_id', 'doc2vec_300', 'quality']]

vector_columns = pd.DataFrame(df_300['doc2vec_300'].tolist(), columns=[f'var_{i+1}' for i in range(300)])
df_300 = df_300.join(vector_columns)
df_300

,title,page_id,doc2vec_300,quality,var_1,var_2,var_3,var_4,var_5,var_6,...,var_291,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300
0,Mayan languages,182013,"[-1.2080181, -0.70352554, -1.3096839, 0.218799...",1,-1.208018,-0.703526,-1.309684,0.218800,0.101862,-0.049087,...,-0.489064,0.333579,-0.934439,-0.035961,1.735480,-0.659636,-0.295216,-0.043667,1.033683,-0.054707
1,Mu'awiya I,207068,"[0.1324497, -0.15933883, -0.56959057, 0.640978...",1,0.132450,-0.159339,-0.569591,0.640979,0.336653,-1.091571,...,-0.884316,0.820414,-0.608468,-0.848112,0.243184,-0.785824,-0.588538,0.179895,0.434624,0.929594
2,The Fountainhead,180464,"[-0.39575249, -0.272289246, -0.45062992, -1.13...",1,-0.395752,-0.272289,-0.450630,-1.139561,0.334936,-0.620625,...,-0.473762,-0.038812,-0.151250,-0.355493,1.358944,0.819606,0.112307,-0.171289,-0.305248,-0.778654
3,Northern pintail,218361,"[-0.15325828, -1.0265552, -0.950474, -0.276538...",1,-0.153258,-1.026555,-0.950474,-0.276538,-0.450867,0.937326,...,-0.377420,0.533450,-0.704751,-0.036939,0.444568,0.897536,-0.457526,-0.183522,0.231193,-0.619215
4,Manhattan Project,19603,"[0.185518786, 0.867399991, -0.350051463, 0.414...",1,0.185519,0.867400,-0.350051,0.414028,-0.081832,0.457686,...,-0.800296,0.640362,0.470449,-0.097818,0.380964,0.502427,-0.000494,-0.053854,-0.081947,-1.170469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,"[-0.584085286, 0.0687814727, -0.303456843, -0....",0,-0.584085,0.068781,-0.303457,-0.074142,-0.264985,0.007812,...,-0.570415,0.184842,0.262775,-0.104835,0.307361,0.174740,-0.003305,-0.043815,0.241955,-0.099139
5196,Minawara and Multultu,95240,"[-0.1366955, -0.18754531, -0.12493268, -0.0181...",0,-0.136695,-0.187545,-0.124933,-0.018172,-0.221326,0.095070,...,-0.180165,0.178297,-0.142925,-0.145432,0.291195,0.013964,-0.125530,0.028248,-0.010183,-0.103178
5197,Theophylline/ephedra/hydroxyzine,262652,"[-0.196540684, -0.0682713836, -0.128727436, 0....",0,-0.196541,-0.068271,-0.128727,0.048658,-0.219939,0.084227,...,-0.249470,0.037468,-0.185873,-0.037281,0.186688,0.053862,-0.047947,0.102050,0.092996,-0.018611
5198,"Channel Lake, Illinois",111450,"[-0.37194127, -0.10045881, -0.15332541, -0.275...",0,-0.371941,-0.100459,-0.153325,-0.275344,-0.234574,-0.023475,...,-0.593767,0.296616,-0.559921,0.085608,0.307723,0.328705,0.377227,0.204556,-0.016480,-0.175796


In [39]:
# Combine all text features with doc2vec_300
combined_300 = text.merge(df_300, on='title')
combined_300

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_291,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.489064,0.333579,-0.934439,-0.035961,1.735480,-0.659636,-0.295216,-0.043667,1.033683,-0.054707
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,-0.884316,0.820414,-0.608468,-0.848112,0.243184,-0.785824,-0.588538,0.179895,0.434624,0.929594
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.473762,-0.038812,-0.151250,-0.355493,1.358944,0.819606,0.112307,-0.171289,-0.305248,-0.778654
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-0.377420,0.533450,-0.704751,-0.036939,0.444568,0.897536,-0.457526,-0.183522,0.231193,-0.619215
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.800296,0.640362,0.470449,-0.097818,0.380964,0.502427,-0.000494,-0.053854,-0.081947,-1.170469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.570415,0.184842,0.262775,-0.104835,0.307361,0.174740,-0.003305,-0.043815,0.241955,-0.099139
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.180165,0.178297,-0.142925,-0.145432,0.291195,0.013964,-0.125530,0.028248,-0.010183,-0.103178
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.249470,0.037468,-0.185873,-0.037281,0.186688,0.053862,-0.047947,0.102050,0.092996,-0.018611
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.593767,0.296616,-0.559921,0.085608,0.307723,0.328705,0.377227,0.204556,-0.016480,-0.175796


In [40]:
combined_300[combined_300['page_id_x']!=combined_300['page_id_y']]

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_291,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300


In [41]:
combined_300.drop(['page_id_y', 'quality_y', 'doc2vec_300'], axis=1, inplace=True)
combined_300.rename(columns={'page_id_x': 'page_id', 'quality_x': 'quality'}, inplace=True)
combined_300

,title,page_id,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_291,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,-0.489064,0.333579,-0.934439,-0.035961,1.735480,-0.659636,-0.295216,-0.043667,1.033683,-0.054707
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,-0.884316,0.820414,-0.608468,-0.848112,0.243184,-0.785824,-0.588538,0.179895,0.434624,0.929594
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,-0.473762,-0.038812,-0.151250,-0.355493,1.358944,0.819606,0.112307,-0.171289,-0.305248,-0.778654
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,-0.377420,0.533450,-0.704751,-0.036939,0.444568,0.897536,-0.457526,-0.183522,0.231193,-0.619215
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.800296,0.640362,0.470449,-0.097818,0.380964,0.502427,-0.000494,-0.053854,-0.081947,-1.170469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.570415,0.184842,0.262775,-0.104835,0.307361,0.174740,-0.003305,-0.043815,0.241955,-0.099139
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.180165,0.178297,-0.142925,-0.145432,0.291195,0.013964,-0.125530,0.028248,-0.010183,-0.103178
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.249470,0.037468,-0.185873,-0.037281,0.186688,0.053862,-0.047947,0.102050,0.092996,-0.018611
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.593767,0.296616,-0.559921,0.085608,0.307723,0.328705,0.377227,0.204556,-0.016480,-0.175796


In [43]:
X = combined_300.drop(['title', 'page_id', 'quality'], axis=1)
y = combined_300['quality']

result_300 = train_and_evaluate_models(X, y)
result_300

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.870192,0.870633,0.870192,0.870156,0.883654,0.883723,0.883654,0.883649
1,SVM,0.893990,0.895177,0.893990,0.893911,0.906731,0.908000,0.906731,0.906658
2,Random Forest,0.880048,0.882111,0.877404,0.882979,0.886538,0.887829,0.886538,0.886444
3,AdaBoost,0.878125,0.878548,0.878125,0.878090,0.874038,0.874439,0.874038,0.874005
4,XGBoost,0.903125,0.904164,0.903125,0.903065,0.903846,0.904229,0.903846,0.903823
5,MLP,0.886298,0.885343,0.886779,0.882426,0.889423,0.889597,0.889423,0.889411
6,KNN,0.845192,0.849387,0.845192,0.844723,0.849038,0.852165,0.849038,0.848703


__Combining all text features and doc2vec_500__

In [44]:
# Create features using doc2vec_500
df_500 = doc2vec.copy()
df_500 = df_500[['title', 'page_id', 'doc2vec_500', 'quality']]

vector_columns = pd.DataFrame(df_500['doc2vec_500'].tolist(), columns=[f'var_{i+1}' for i in range(500)])
df_500 = df_500.join(vector_columns)
df_500

,title,page_id,doc2vec_500,quality,var_1,var_2,var_3,var_4,var_5,var_6,...,var_491,var_492,var_493,var_494,var_495,var_496,var_497,var_498,var_499,var_500
0,Mayan languages,182013,"[-0.108123168, -0.505179107, -0.473065108, 0.9...",1,-0.108123,-0.505179,-0.473065,0.931556,0.209105,0.321946,...,0.056441,-0.715848,1.087974,-0.967918,-0.329986,0.395207,-0.404756,0.465462,0.049110,-0.698453
1,Mu'awiya I,207068,"[-1.1343669, -0.30720899, 0.24345188, 0.244366...",1,-1.134367,-0.307209,0.243452,0.244366,0.079750,0.535138,...,-0.255762,0.375592,0.349120,0.477647,-0.102216,-0.106009,-0.254864,0.372623,-0.534829,-0.388574
2,The Fountainhead,180464,"[0.451237231, -0.393101126, 0.152836084, -0.30...",1,0.451237,-0.393101,0.152836,-0.309696,-0.370821,0.415327,...,0.312036,0.382692,-0.542440,-1.167028,0.141248,0.380896,-0.848125,-0.040682,-0.012778,-0.219372
3,Northern pintail,218361,"[-0.255974084, -1.00488138, -0.753050864, 0.16...",1,-0.255974,-1.004881,-0.753051,0.161540,-0.036913,0.318206,...,0.074718,-0.191207,0.389778,-0.570847,-0.056948,0.005303,-0.050988,0.393957,-0.347253,-0.227030
4,Manhattan Project,19603,"[-0.00231942884, -0.461694479, 0.310965121, 0....",1,-0.002319,-0.461694,0.310965,0.462390,0.169759,0.416137,...,-0.131481,0.498386,0.291432,0.096579,-0.637454,0.089752,-0.867878,-0.223389,-0.691802,-0.127201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,"[-0.130619839, -0.123517677, 0.00214104261, 0....",0,-0.130620,-0.123518,0.002141,0.163061,-0.009361,0.186305,...,-0.025766,0.221403,0.120302,-0.591961,0.177544,0.488346,-0.058395,0.065176,0.086197,-0.313442
5196,Minawara and Multultu,95240,"[-0.155988351, -0.0939027667, -0.0548811592, 0...",0,-0.155988,-0.093903,-0.054881,0.055432,0.034402,0.179713,...,-0.047278,0.093827,0.220350,-0.120718,-0.309029,0.190342,-0.129211,-0.057561,-0.040555,-0.134973
5197,Theophylline/ephedra/hydroxyzine,262652,"[-0.05836337, 0.03332838, -0.02910833, 0.03826...",0,-0.058363,0.033328,-0.029108,0.038263,0.090555,0.112414,...,-0.118516,0.037691,0.066726,-0.050097,-0.080435,0.136775,-0.049201,0.104074,-0.046042,-0.157075
5198,"Channel Lake, Illinois",111450,"[-0.161765054, 0.0333138444, -0.081770055, 0.1...",0,-0.161765,0.033314,-0.081770,0.143502,0.056106,0.097462,...,-0.058019,0.250781,0.227399,-0.208683,-0.309399,0.237372,0.073670,0.031651,-0.037014,0.135632


In [45]:
# Combine all text features with doc2vec_500
combined_500 = text.merge(df_500, on='title')
combined_500

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_491,var_492,var_493,var_494,var_495,var_496,var_497,var_498,var_499,var_500
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,0.056441,-0.715848,1.087974,-0.967918,-0.329986,0.395207,-0.404756,0.465462,0.049110,-0.698453
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,-0.255762,0.375592,0.349120,0.477647,-0.102216,-0.106009,-0.254864,0.372623,-0.534829,-0.388574
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,0.312036,0.382692,-0.542440,-1.167028,0.141248,0.380896,-0.848125,-0.040682,-0.012778,-0.219372
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,0.074718,-0.191207,0.389778,-0.570847,-0.056948,0.005303,-0.050988,0.393957,-0.347253,-0.227030
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.131481,0.498386,0.291432,0.096579,-0.637454,0.089752,-0.867878,-0.223389,-0.691802,-0.127201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.025766,0.221403,0.120302,-0.591961,0.177544,0.488346,-0.058395,0.065176,0.086197,-0.313442
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.047278,0.093827,0.220350,-0.120718,-0.309029,0.190342,-0.129211,-0.057561,-0.040555,-0.134973
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.118516,0.037691,0.066726,-0.050097,-0.080435,0.136775,-0.049201,0.104074,-0.046042,-0.157075
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.058019,0.250781,0.227399,-0.208683,-0.309399,0.237372,0.073670,0.031651,-0.037014,0.135632


In [46]:
combined_500[combined_500['page_id_x']!=combined_500['page_id_y']]

,title,page_id_x,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_491,var_492,var_493,var_494,var_495,var_496,var_497,var_498,var_499,var_500


In [47]:
combined_500.drop(['page_id_y', 'quality_y', 'doc2vec_500'], axis=1, inplace=True)
combined_500.rename(columns={'page_id_x': 'page_id', 'quality_x': 'quality'}, inplace=True)
combined_500

,title,page_id,char_count,word_count,sentence_count,syllable_count,complex_word_count,section_count,subsection_count,paragraph_count,...,var_491,var_492,var_493,var_494,var_495,var_496,var_497,var_498,var_499,var_500
0,Mayan languages,182013,0.554382,0.490641,0.350556,0.563203,0.727204,0.313583,1.614782,0.765594,...,0.056441,-0.715848,1.087974,-0.967918,-0.329986,0.395207,-0.404756,0.465462,0.049110,-0.698453
1,Mu'awiya I,207068,1.902108,1.915008,1.614977,1.920173,1.860678,0.660305,0.804225,0.622956,...,-0.255762,0.375592,0.349120,0.477647,-0.102216,-0.106009,-0.254864,0.372623,-0.534829,-0.388574
2,The Fountainhead,180464,0.527779,0.520419,0.849882,0.568074,0.666285,0.140222,1.209503,0.123721,...,0.312036,0.382692,-0.542440,-1.167028,0.141248,0.380896,-0.848125,-0.040682,-0.012778,-0.219372
3,Northern pintail,218361,-0.519528,-0.520312,-0.575612,-0.539584,-0.622258,-0.379860,-0.715570,-0.518153,...,0.074718,-0.191207,0.389778,-0.570847,-0.056948,0.005303,-0.050988,0.393957,-0.347253,-0.227030
4,Manhattan Project,19603,3.393837,3.347316,3.580064,3.595369,4.328846,2.567274,2.222700,2.358391,...,-0.131481,0.498386,0.291432,0.096579,-0.637454,0.089752,-0.867878,-0.223389,-0.691802,-0.127201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,-1.022361,-1.048866,-1.042723,-1.020531,-0.919472,-1.420025,-0.918209,-0.993615,...,-0.025766,0.221403,0.120302,-0.591961,0.177544,0.488346,-0.058395,0.065176,0.086197,-0.313442
5196,Minawara and Multultu,95240,-1.089358,-1.108422,-1.074937,-1.095216,-1.028389,-1.593386,-0.918209,-1.136253,...,-0.047278,0.093827,0.220350,-0.120718,-0.309029,0.190342,-0.129211,-0.057561,-0.040555,-0.134973
5197,Theophylline/ephedra/hydroxyzine,262652,-1.095542,-1.115618,-1.082991,-1.101530,-1.032081,-1.593386,-0.918209,-1.136253,...,-0.118516,0.037691,0.066726,-0.050097,-0.080435,0.136775,-0.049201,0.104074,-0.046042,-0.157075
5198,"Channel Lake, Illinois",111450,-1.004166,-1.005192,-0.986347,-1.010428,-0.971161,-1.246665,-0.715570,-0.993615,...,-0.058019,0.250781,0.227399,-0.208683,-0.309399,0.237372,0.073670,0.031651,-0.037014,0.135632


In [48]:
X = combined_500.drop(['title', 'page_id', 'quality'], axis=1)
y = combined_500['quality']

result_500 = train_and_evaluate_models(X, y)
result_500

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.856731,0.857101,0.856731,0.856693,0.862500,0.862534,0.862500,0.862497
1,SVM,0.894712,0.895945,0.894712,0.894629,0.906731,0.909032,0.906731,0.906599
2,Random Forest,0.871154,0.879310,0.870913,0.874937,0.885577,0.887758,0.885577,0.885416
3,AdaBoost,0.872115,0.872752,0.872115,0.872062,0.875000,0.875089,0.875000,0.874993
4,XGBoost,0.898077,0.898850,0.898077,0.898030,0.902885,0.903316,0.902885,0.902859
5,MLP,0.876442,0.875851,0.874038,0.878069,0.896154,0.896629,0.896154,0.896123
6,KNN,0.811058,0.827126,0.811058,0.808713,0.825000,0.836468,0.825000,0.823496


### Combining selected text features and doc2vec

In [56]:
structure_style_edit = text.iloc[:, list(range(0,1))+list(range(7, 31))+list(range(38, 48)) + [-1]]

__Combining selected text features and doc2vec_20__

In [57]:
# Combine selected text features with doc2vec_20
combined2_20 = structure_style_edit.merge(df_20, on='title')
combined2_20

,title,section_count,subsection_count,paragraph_count,mean_section_size,mean_paragraph_size,longest_section_size,shortest_section_size,abstract_size,abstract_text_ratio,...,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20
0,Mayan languages,0.313583,1.614782,0.765594,0.588225,-0.116747,-0.082148,0.040986,0.498716,-0.390911,...,2.589589,-3.304577,-0.880189,0.132373,-1.697009,-1.738696,0.542941,0.903049,-1.734197,0.417213
1,Mu'awiya I,0.660305,0.804225,0.622956,1.578149,1.859600,0.538504,0.765795,2.003344,-0.399434,...,1.648793,0.241381,-0.366885,-0.707308,-2.055417,-1.595411,1.199849,-2.256915,-1.318837,-0.628533
2,The Fountainhead,0.140222,1.209503,0.123721,0.688174,0.869100,-0.006314,-0.307216,0.852630,-0.344703,...,-0.990515,0.894514,-0.850906,-1.573790,-0.939912,0.271126,0.087762,0.493070,-0.744864,0.943934
3,Northern pintail,-0.379860,-0.715570,-0.518153,-0.356550,0.015031,-0.731613,0.509226,0.092407,-0.092585,...,-0.217106,-0.038556,-2.535328,-1.079626,0.984010,-3.052293,-1.283746,0.150716,-2.270509,0.511353
4,Manhattan Project,2.567274,2.222700,2.358391,1.168441,0.879893,1.278631,0.241660,1.589126,-0.493332,...,-0.649519,1.573444,0.557071,-1.295349,1.095210,-0.513653,-0.424813,0.191953,-2.206298,-0.896591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,-1.420025,-0.918209,-0.993615,-1.258838,-0.956394,-1.026748,-0.586693,0.391949,3.734560,...,1.364483,1.565599,-0.117768,-0.553074,-1.408595,-1.481983,0.412338,-0.264874,-1.365068,-1.363219
5196,Minawara and Multultu,-1.593386,-0.918209,-1.136253,-1.258838,-0.291067,-1.026748,-0.586693,-1.102794,3.721017,...,-0.029130,0.056235,-1.061213,0.044772,-0.044322,-0.851839,-0.056814,0.286022,-0.846064,-0.032695
5197,Theophylline/ephedra/hydroxyzine,-1.593386,-0.918209,-1.136253,-1.258838,-0.914731,-1.026748,-0.586693,-1.240207,3.721017,...,0.014874,0.138150,-0.292356,-0.286313,-0.223987,-0.202073,-0.371795,0.419180,-0.500988,-0.126529
5198,"Channel Lake, Illinois",-1.246665,-0.715570,-0.993615,-0.802396,-0.686546,-0.825651,-0.062558,-1.338077,-0.287728,...,1.413022,1.097628,-1.258792,0.260787,-0.695690,-1.898501,-1.764783,0.649597,-1.145714,-0.401934


In [59]:
combined2_20.drop(['quality_y', 'doc2vec_20'], axis=1, inplace=True)
combined2_20.rename(columns={'quality_x': 'quality'}, inplace=True)
combined2_20

,title,section_count,subsection_count,paragraph_count,mean_section_size,mean_paragraph_size,longest_section_size,shortest_section_size,abstract_size,abstract_text_ratio,...,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20
0,Mayan languages,0.313583,1.614782,0.765594,0.588225,-0.116747,-0.082148,0.040986,0.498716,-0.390911,...,2.589589,-3.304577,-0.880189,0.132373,-1.697009,-1.738696,0.542941,0.903049,-1.734197,0.417213
1,Mu'awiya I,0.660305,0.804225,0.622956,1.578149,1.859600,0.538504,0.765795,2.003344,-0.399434,...,1.648793,0.241381,-0.366885,-0.707308,-2.055417,-1.595411,1.199849,-2.256915,-1.318837,-0.628533
2,The Fountainhead,0.140222,1.209503,0.123721,0.688174,0.869100,-0.006314,-0.307216,0.852630,-0.344703,...,-0.990515,0.894514,-0.850906,-1.573790,-0.939912,0.271126,0.087762,0.493070,-0.744864,0.943934
3,Northern pintail,-0.379860,-0.715570,-0.518153,-0.356550,0.015031,-0.731613,0.509226,0.092407,-0.092585,...,-0.217106,-0.038556,-2.535328,-1.079626,0.984010,-3.052293,-1.283746,0.150716,-2.270509,0.511353
4,Manhattan Project,2.567274,2.222700,2.358391,1.168441,0.879893,1.278631,0.241660,1.589126,-0.493332,...,-0.649519,1.573444,0.557071,-1.295349,1.095210,-0.513653,-0.424813,0.191953,-2.206298,-0.896591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,-1.420025,-0.918209,-0.993615,-1.258838,-0.956394,-1.026748,-0.586693,0.391949,3.734560,...,1.364483,1.565599,-0.117768,-0.553074,-1.408595,-1.481983,0.412338,-0.264874,-1.365068,-1.363219
5196,Minawara and Multultu,-1.593386,-0.918209,-1.136253,-1.258838,-0.291067,-1.026748,-0.586693,-1.102794,3.721017,...,-0.029130,0.056235,-1.061213,0.044772,-0.044322,-0.851839,-0.056814,0.286022,-0.846064,-0.032695
5197,Theophylline/ephedra/hydroxyzine,-1.593386,-0.918209,-1.136253,-1.258838,-0.914731,-1.026748,-0.586693,-1.240207,3.721017,...,0.014874,0.138150,-0.292356,-0.286313,-0.223987,-0.202073,-0.371795,0.419180,-0.500988,-0.126529
5198,"Channel Lake, Illinois",-1.246665,-0.715570,-0.993615,-0.802396,-0.686546,-0.825651,-0.062558,-1.338077,-0.287728,...,1.413022,1.097628,-1.258792,0.260787,-0.695690,-1.898501,-1.764783,0.649597,-1.145714,-0.401934


In [60]:
X = combined2_20.drop(['title', 'page_id', 'quality'], axis=1)
y = combined2_20['quality']

result2_20 = train_and_evaluate_models(X, y)
result2_20

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.887500,0.887851,0.887500,0.887474,0.905769,0.905985,0.905769,0.905757
1,SVM,0.899760,0.900678,0.899760,0.899702,0.908654,0.909929,0.908654,0.908583
2,Random Forest,0.895192,0.898533,0.896154,0.897531,0.901923,0.902405,0.901923,0.901894
3,AdaBoost,0.890385,0.890962,0.890385,0.890343,0.883654,0.883667,0.883654,0.883653
4,XGBoost,0.912260,0.912991,0.912260,0.912222,0.913462,0.913682,0.913462,0.913450
5,MLP,0.892308,0.894928,0.894712,0.893723,0.897115,0.897152,0.897115,0.897113
6,KNN,0.857452,0.860536,0.857452,0.857146,0.875962,0.877875,0.875962,0.875804


__Combining selected text features and doc2vec_50__

In [61]:
# Combine selected text features with doc2vec_50
combined2_50 = structure_style_edit.merge(df_50, on='title')
combined2_50

,title,section_count,subsection_count,paragraph_count,mean_section_size,mean_paragraph_size,longest_section_size,shortest_section_size,abstract_size,abstract_text_ratio,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,Mayan languages,0.313583,1.614782,0.765594,0.588225,-0.116747,-0.082148,0.040986,0.498716,-0.390911,...,-0.609255,1.594196,1.374301,0.580563,0.753562,-0.446759,1.098445,2.162488,0.269427,-2.083542
1,Mu'awiya I,0.660305,0.804225,0.622956,1.578149,1.859600,0.538504,0.765795,2.003344,-0.399434,...,0.664694,-0.060651,0.799270,-0.958273,0.337148,1.416505,2.117402,-0.944979,2.191666,-0.045392
2,The Fountainhead,0.140222,1.209503,0.123721,0.688174,0.869100,-0.006314,-0.307216,0.852630,-0.344703,...,0.899741,-0.025535,0.509132,-1.556558,-1.069190,0.774159,0.017817,1.039071,0.768530,-1.704601
3,Northern pintail,-0.379860,-0.715570,-0.518153,-0.356550,0.015031,-0.731613,0.509226,0.092407,-0.092585,...,0.042587,-0.301828,-0.268440,-1.096084,0.953988,1.231612,0.365509,0.223079,0.547631,-0.617974
4,Manhattan Project,2.567274,2.222700,2.358391,1.168441,0.879893,1.278631,0.241660,1.589126,-0.493332,...,0.606108,1.389589,1.016950,-0.579469,0.645308,1.044399,-0.144792,1.077177,-1.111737,-1.869863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,-1.420025,-0.918209,-0.993615,-1.258838,-0.956394,-1.026748,-0.586693,0.391949,3.734560,...,-0.075037,-0.018248,0.332292,-0.523666,-0.004916,0.925149,-0.161927,0.669919,-0.779033,-0.517922
5196,Minawara and Multultu,-1.593386,-0.918209,-1.136253,-1.258838,-0.291067,-1.026748,-0.586693,-1.102794,3.721017,...,0.147598,-0.126686,0.140621,-0.720846,0.455311,-0.059910,0.031989,0.164125,0.189279,-0.170787
5197,Theophylline/ephedra/hydroxyzine,-1.593386,-0.918209,-1.136253,-1.258838,-0.914731,-1.026748,-0.586693,-1.240207,3.721017,...,0.037891,-0.399214,0.291244,-0.123542,0.090050,-0.166694,-0.044172,0.010083,0.038130,-0.616626
5198,"Channel Lake, Illinois",-1.246665,-0.715570,-0.993615,-0.802396,-0.686546,-0.825651,-0.062558,-1.338077,-0.287728,...,-0.256851,-0.634215,-0.124935,-0.202607,0.482201,-0.721821,-0.682262,0.804415,0.242318,-1.592898


In [62]:
combined2_50.drop(['quality_y', 'doc2vec_50'], axis=1, inplace=True)
combined2_50.rename(columns={'quality_x': 'quality'}, inplace=True)
combined2_50

,title,section_count,subsection_count,paragraph_count,mean_section_size,mean_paragraph_size,longest_section_size,shortest_section_size,abstract_size,abstract_text_ratio,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,Mayan languages,0.313583,1.614782,0.765594,0.588225,-0.116747,-0.082148,0.040986,0.498716,-0.390911,...,-0.609255,1.594196,1.374301,0.580563,0.753562,-0.446759,1.098445,2.162488,0.269427,-2.083542
1,Mu'awiya I,0.660305,0.804225,0.622956,1.578149,1.859600,0.538504,0.765795,2.003344,-0.399434,...,0.664694,-0.060651,0.799270,-0.958273,0.337148,1.416505,2.117402,-0.944979,2.191666,-0.045392
2,The Fountainhead,0.140222,1.209503,0.123721,0.688174,0.869100,-0.006314,-0.307216,0.852630,-0.344703,...,0.899741,-0.025535,0.509132,-1.556558,-1.069190,0.774159,0.017817,1.039071,0.768530,-1.704601
3,Northern pintail,-0.379860,-0.715570,-0.518153,-0.356550,0.015031,-0.731613,0.509226,0.092407,-0.092585,...,0.042587,-0.301828,-0.268440,-1.096084,0.953988,1.231612,0.365509,0.223079,0.547631,-0.617974
4,Manhattan Project,2.567274,2.222700,2.358391,1.168441,0.879893,1.278631,0.241660,1.589126,-0.493332,...,0.606108,1.389589,1.016950,-0.579469,0.645308,1.044399,-0.144792,1.077177,-1.111737,-1.869863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,-1.420025,-0.918209,-0.993615,-1.258838,-0.956394,-1.026748,-0.586693,0.391949,3.734560,...,-0.075037,-0.018248,0.332292,-0.523666,-0.004916,0.925149,-0.161927,0.669919,-0.779033,-0.517922
5196,Minawara and Multultu,-1.593386,-0.918209,-1.136253,-1.258838,-0.291067,-1.026748,-0.586693,-1.102794,3.721017,...,0.147598,-0.126686,0.140621,-0.720846,0.455311,-0.059910,0.031989,0.164125,0.189279,-0.170787
5197,Theophylline/ephedra/hydroxyzine,-1.593386,-0.918209,-1.136253,-1.258838,-0.914731,-1.026748,-0.586693,-1.240207,3.721017,...,0.037891,-0.399214,0.291244,-0.123542,0.090050,-0.166694,-0.044172,0.010083,0.038130,-0.616626
5198,"Channel Lake, Illinois",-1.246665,-0.715570,-0.993615,-0.802396,-0.686546,-0.825651,-0.062558,-1.338077,-0.287728,...,-0.256851,-0.634215,-0.124935,-0.202607,0.482201,-0.721821,-0.682262,0.804415,0.242318,-1.592898


In [63]:
X = combined2_50.drop(['title', 'page_id', 'quality'], axis=1)
y = combined2_50['quality']

result2_50 = train_and_evaluate_models(X, y)
result2_50

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.889183,0.889570,0.889183,0.889155,0.906731,0.907069,0.906731,0.906711
1,SVM,0.904327,0.905327,0.904327,0.904266,0.912500,0.913174,0.912500,0.912464
2,Random Forest,0.893029,0.894874,0.896635,0.893412,0.891346,0.891765,0.891346,0.891317
3,AdaBoost,0.888942,0.889831,0.888942,0.888876,0.887500,0.887742,0.887500,0.887482
4,XGBoost,0.906731,0.907637,0.906731,0.906678,0.898077,0.898554,0.898077,0.898046
5,MLP,0.901683,0.896868,0.902644,0.900699,0.907692,0.907789,0.907692,0.907687
6,KNN,0.865385,0.866818,0.865385,0.865258,0.878846,0.880111,0.878846,0.878745


__Combining selected text features and doc2vec_100__

In [64]:
# Combine selected text features with doc2vec_100
combined2_100 = structure_style_edit.merge(df_100, on='title')
combined2_100

,title,section_count,subsection_count,paragraph_count,mean_section_size,mean_paragraph_size,longest_section_size,shortest_section_size,abstract_size,abstract_text_ratio,...,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100
0,Mayan languages,0.313583,1.614782,0.765594,0.588225,-0.116747,-0.082148,0.040986,0.498716,-0.390911,...,-0.780045,0.126435,0.457684,-0.976941,-0.033570,0.909304,-0.341177,-0.686826,0.434214,-0.085368
1,Mu'awiya I,0.660305,0.804225,0.622956,1.578149,1.859600,0.538504,0.765795,2.003344,-0.399434,...,-0.221565,1.052404,-0.155383,0.833835,-0.108549,2.475773,1.293293,-0.061450,0.716968,0.258674
2,The Fountainhead,0.140222,1.209503,0.123721,0.688174,0.869100,-0.006314,-0.307216,0.852630,-0.344703,...,-0.328424,0.488366,-1.207654,0.679866,0.896923,-0.140029,-0.371535,-0.175522,1.484022,-0.286713
3,Northern pintail,-0.379860,-0.715570,-0.518153,-0.356550,0.015031,-0.731613,0.509226,0.092407,-0.092585,...,-0.514490,0.849899,-0.754519,-1.038317,0.780983,-1.046274,-1.555287,-0.416799,0.093230,-0.191590
4,Manhattan Project,2.567274,2.222700,2.358391,1.168441,0.879893,1.278631,0.241660,1.589126,-0.493332,...,-0.186288,-0.263700,0.289110,0.183705,-0.029313,0.589104,-1.494260,-0.030581,0.671126,2.263582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,-1.420025,-0.918209,-0.993615,-1.258838,-0.956394,-1.026748,-0.586693,0.391949,3.734560,...,-0.559690,0.546811,-0.635086,0.178669,0.045058,0.377926,0.065588,-0.109098,1.108363,0.593977
5196,Minawara and Multultu,-1.593386,-0.918209,-1.136253,-1.258838,-0.291067,-1.026748,-0.586693,-1.102794,3.721017,...,-0.222619,0.109795,-0.336194,-0.337256,-0.021674,0.307858,-0.396411,-0.137848,0.027160,-0.120415
5197,Theophylline/ephedra/hydroxyzine,-1.593386,-0.918209,-1.136253,-1.258838,-0.914731,-1.026748,-0.586693,-1.240207,3.721017,...,-0.165442,-0.281676,-0.094921,-0.101845,-0.039904,0.236989,-0.154063,-0.092468,0.126725,0.089715
5198,"Channel Lake, Illinois",-1.246665,-0.715570,-0.993615,-0.802396,-0.686546,-0.825651,-0.062558,-1.338077,-0.287728,...,0.176425,0.115690,-0.924295,-0.120078,-0.240744,0.456143,-0.160728,-0.819485,0.037214,0.470418


In [65]:
combined2_100.drop(['quality_y', 'doc2vec_100'], axis=1, inplace=True)
combined2_100.rename(columns={'quality_x': 'quality'}, inplace=True)
combined2_100

,title,section_count,subsection_count,paragraph_count,mean_section_size,mean_paragraph_size,longest_section_size,shortest_section_size,abstract_size,abstract_text_ratio,...,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100
0,Mayan languages,0.313583,1.614782,0.765594,0.588225,-0.116747,-0.082148,0.040986,0.498716,-0.390911,...,-0.780045,0.126435,0.457684,-0.976941,-0.033570,0.909304,-0.341177,-0.686826,0.434214,-0.085368
1,Mu'awiya I,0.660305,0.804225,0.622956,1.578149,1.859600,0.538504,0.765795,2.003344,-0.399434,...,-0.221565,1.052404,-0.155383,0.833835,-0.108549,2.475773,1.293293,-0.061450,0.716968,0.258674
2,The Fountainhead,0.140222,1.209503,0.123721,0.688174,0.869100,-0.006314,-0.307216,0.852630,-0.344703,...,-0.328424,0.488366,-1.207654,0.679866,0.896923,-0.140029,-0.371535,-0.175522,1.484022,-0.286713
3,Northern pintail,-0.379860,-0.715570,-0.518153,-0.356550,0.015031,-0.731613,0.509226,0.092407,-0.092585,...,-0.514490,0.849899,-0.754519,-1.038317,0.780983,-1.046274,-1.555287,-0.416799,0.093230,-0.191590
4,Manhattan Project,2.567274,2.222700,2.358391,1.168441,0.879893,1.278631,0.241660,1.589126,-0.493332,...,-0.186288,-0.263700,0.289110,0.183705,-0.029313,0.589104,-1.494260,-0.030581,0.671126,2.263582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,-1.420025,-0.918209,-0.993615,-1.258838,-0.956394,-1.026748,-0.586693,0.391949,3.734560,...,-0.559690,0.546811,-0.635086,0.178669,0.045058,0.377926,0.065588,-0.109098,1.108363,0.593977
5196,Minawara and Multultu,-1.593386,-0.918209,-1.136253,-1.258838,-0.291067,-1.026748,-0.586693,-1.102794,3.721017,...,-0.222619,0.109795,-0.336194,-0.337256,-0.021674,0.307858,-0.396411,-0.137848,0.027160,-0.120415
5197,Theophylline/ephedra/hydroxyzine,-1.593386,-0.918209,-1.136253,-1.258838,-0.914731,-1.026748,-0.586693,-1.240207,3.721017,...,-0.165442,-0.281676,-0.094921,-0.101845,-0.039904,0.236989,-0.154063,-0.092468,0.126725,0.089715
5198,"Channel Lake, Illinois",-1.246665,-0.715570,-0.993615,-0.802396,-0.686546,-0.825651,-0.062558,-1.338077,-0.287728,...,0.176425,0.115690,-0.924295,-0.120078,-0.240744,0.456143,-0.160728,-0.819485,0.037214,0.470418


In [66]:
X = combined2_100.drop(['title', 'page_id', 'quality'], axis=1)
y = combined2_100['quality']

result2_100 = train_and_evaluate_models(X, y)
result2_100

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.887981,0.888393,0.887981,0.887949,0.907692,0.907988,0.907692,0.907676
1,SVM,0.897837,0.899143,0.897837,0.897750,0.909615,0.910642,0.909615,0.909559
2,Random Forest,0.889423,0.893637,0.891587,0.890291,0.898077,0.898667,0.898077,0.898039
3,AdaBoost,0.882692,0.883598,0.882692,0.882617,0.883654,0.884064,0.883654,0.883623
4,XGBoost,0.904808,0.905655,0.904808,0.904758,0.906731,0.907166,0.906731,0.906706
5,MLP,0.893750,0.895395,0.896394,0.894421,0.899038,0.899052,0.899038,0.899038
6,KNN,0.860817,0.860955,0.860817,0.860804,0.878846,0.878852,0.878846,0.878846
